# **Feature engineering my crypto data**

In this *notebook*, I will add some important *features* to my data, that initially has only *prices* (close, high, low and open) and *volume*, which doesn't offer that much info to train a model to predict future prices.

In [2]:
import pandas as pd

Let's load and have a peak into our data

In [8]:
btc = pd.read_csv('coins_data/btc.csv')
eth = pd.read_csv('coins_data/eth.csv')
sol = pd.read_csv('coins_data/sol.csv')
bnb = pd.read_csv('coins_data/bnb.csv')
btc.head(10)

,Price,Close,High,Low,Open,Volume
0,Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
1,Date,NaN,NaN,NaN,NaN,NaN
2,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
3,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
4,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
5,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600
6,2014-09-21,398.8210144042969,412.4259948730469,393.1809997558594,408.0849914550781,26580100
7,2014-09-22,402.1520080566406,406.9159851074219,397.1300048828125,399.1000061035156,24127600
8,2014-09-23,435.7909851074219,441.5570068359375,396.1969909667969,402.0920104980469,45099500
9,2014-09-24,423.2049865722656,436.11199951171875,421.1319885253906,435.7510070800781,30627700


## **1. Moving averages (MA)**

The first feature we'll 'engineer', is moving averages.

Moving averages in crypto trading smooth out price data to identify trends by calculating the average price over a specific period. They help traders spot potential buy/sell signals, support/resistance levels, and trend reversals.

### **a. 3-day MA**

We'll start with a 3 day moving average, 

## **2.**